In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Image Classification with PyTorch (CNN)

In [2]:
import time
import datetime
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from IPython.display import display

## Declaring Dataloader [load data in NN a/c to batch size (if given)]

In [3]:
ris = resize_image_size = 64
train_data_path="/home/garvit/work_area/zone/Anaconda_WorkSpace/jupyter/PyTorch practice/PyTorch Book/chapter2/dataset/training_set/"
test_data_path="/home/garvit/work_area/zone/Anaconda_WorkSpace/jupyter/PyTorch practice/PyTorch Book/chapter2/dataset/test_set/"

# declaring some trasformations on images for NN feed i.e Scaling images, converting into tensors e.t.c
transformations = transforms.Compose([transforms.Resize((ris,ris)),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean= [0.485, 0.456, 0.406],
                                                      std = [0.229, 0.224, 0.225])])

# applying above transformations
train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=transformations)
# val_data   = torchvision.datasets.ImageFolder(root = val_data_path, transform=transforms) 
test_data  = torchvision.datasets.ImageFolder(root = test_data_path, transform=transformations)


# declaring DataLoader (load data in NN a/c to batch size if given)
BatchSize = 48
train_data_loader = DataLoader(dataset=train_data, batch_size = BatchSize, shuffle=True )
# val_data_loader = DataLoader(dataset=val_data, batch_size = BatchSize)
test_data_loader = DataLoader(dataset=test_data, batch_size = BatchSize, shuffle=True)


## Creating a CNN Network

In [7]:
class CNNNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=5, kernel_size=3, stride=1, padding=1),
            nn.ReLU())
        
        self.classifier = nn.Sequential(
            nn.Linear(64*64*5,4096),
            nn.Linear(4096, 1024),
            nn.Linear(1024,2))        
        
    def forward(self, x):
#         x.view(-1, 3*ris*ris)
        x = self.features(x)
        x = torch.flatten(x,1)
        x = self.classifier(x)
        return x
        

cnnNet = CNNNet()

## Create an optimizer

In [8]:
from torch.optim import Adam, SGD
optimizer = SGD(params=cnnNet.parameters(), lr=0.01)

## Checking for GPU

In [9]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

cnnNet.to(device)

CNNNet(
  (features): Sequential(
    (0): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=20480, out_features=4096, bias=True)
    (1): Linear(in_features=4096, out_features=1024, bias=True)
    (2): Linear(in_features=1024, out_features=2, bias=True)
  )
)

## Defining Training Function 

In [10]:
def train(model, optimiser, loss_fn, train_loader, val_loader, epochs=1, device='cpu'):
    for epoch in range(epochs):
            training_loss = 0
            valid_loss = 0
            model.train()
            for batch in train_loader:
#                 print(f"training batch: {batch}")
                optimizer.zero_grad()
                inputs, target = batch
#                 print(inputs)
                inputs = inputs.to(device)
                target = target.to(device)
                output = model(inputs)
                loss = loss_fn(output, target)
                loss.backward()
                optimizer.step()
                training_loss += loss.data.item()
            training_loss /= len(train_loader.dataset)
            
            model.eval()
            num_correct = 0
            num_examples = 0
            for batch in val_loader:
                inputs, target = batch
                inputs = inputs.to(device)
                output = model(inputs)
                loss = loss_fn(output, target)
                valid_loss += loss.data.item()
                correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], target).view(-1)
                num_correct += torch.sum(correct).item()
                num_examples += correct.shape[0]
            valid_loss /= len(val_loader.dataset)

            print(f'Epoch: {epoch+1},  Training Loss: {round(training_loss,3)}, \
            Validation Loss: {round(valid_loss,3)}, \
            accuracy = {round(num_correct / num_examples,2)}')            
            
    

## Enabling Training

In [11]:
start_time = time.time() 
train(cnnNet, optimizer,nn.CrossEntropyLoss(), train_data_loader,test_data_loader, epochs=5, device=device)
time_took = f"{time.time() - start_time!r}"
time_took = str(datetime.timedelta(seconds=round(float(time_took),2))).rstrip('0')
print(f"Training time: {time_took}")

/home/garvit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1,  Training Loss: 0.014,             Validation Loss: 0.014,             accuracy = 0.57
Epoch: 2,  Training Loss: 0.013,             Validation Loss: 0.013,             accuracy = 0.66
Epoch: 3,  Training Loss: 0.013,             Validation Loss: 0.013,             accuracy = 0.67
Epoch: 4,  Training Loss: 0.012,             Validation Loss: 0.013,             accuracy = 0.65
Epoch: 5,  Training Loss: 0.012,             Validation Loss: 0.013,             accuracy = 0.66
Training time: 0:11:12.68
